In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
#from sqlalchemy.ext.automap import automap_base
#from sqlalchemy.orm import Session

In [2]:
#nrows=10
Grocery_json = pd.read_json('Resources/Grocery_and_Gourmet_Food.json',lines=True)


In [3]:
Grocery_json

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5074155,5,True,"10 4, 2018",AGOPF2VTEOP57,B01HJHSVG6,Bob,"Love this product, very fresh, complete full f...",Exceclent product,1538611200,NaN,NaN,{'Size:': ' 2.29 oz'}
5074156,3,True,"10 3, 2018",A5X5TI4JCH9CN,B01HJHSVG6,Jacqueline Schafer,Good taste but small quantity for the expensiv...,Good taste but small quantity for the price us...,1538524800,NaN,NaN,{'Size:': ' 2.29 oz'}
5074157,4,True,"10 2, 2018",A1WMQQLC3TVAFI,B01HJHSVG6,Motionjunkie,Unlike some of the bad reviews regarding aroma...,See Comments below,1538438400,NaN,NaN,{'Size:': ' 2.29 oz'}
5074158,5,True,"09 30, 2018",ALVP6JZRTEDY6,B01HJHSVG6,Marissa,I absolutely love this vanilla bean paste. I h...,Love!!,1538265600,NaN,NaN,{'Size:': ' 2.29 oz'}


In [4]:
Grocery_json.columns


Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image', 'style'],
      dtype='object')

In [5]:
#read labeled csv
labeled_df = pd.read_csv('Resources/labeled.csv')
labeled_df

,words,sentiment
0,"['comforting', 'know', 'processed', 'like', 's...",pos
1,"['never', 'disappointed', 'cup', 'cider', 'lik...",neg
2,"['never', 'ever', 'write', 'reviews', 'product...",neg
3,"['tasty', 'package', 'arrived', 'neatly', 'pac...",pos
4,"['gonna', 'buy', 'future', 'ate', 'completely'...",pos
...,...,...
809635,"['great', 'tasting', 'tuna', 'easy', 'open', '...",pos
809636,"['addicted', 'product', 'grocer', 'quit', 'car...",neg
809637,"['sure', 'happened', 'box', 'arrived', 'comple...",neg
809638,"['put', 'water', 'bottle', 'done', 'working', ...",pos


In [6]:
#count the number of words in a review
Grocery_json["reviewlength"] = labeled_df["words"].apply(lambda x: len(x))

Grocery_json.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,reviewlength
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN,270.0
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN,287.0
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN,1156.0
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN,126.0
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN,92.0


In [7]:
groceries_df = Grocery_json[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074156,B01HJHSVG6,3,A5X5TI4JCH9CN,"10 3, 2018",1538524800,Good taste but small quantity for the expensiv...,NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [8]:
#drop dupicates
groceries_df = groceries_df.drop_duplicates()
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074156,B01HJHSVG6,3,A5X5TI4JCH9CN,"10 3, 2018",1538524800,Good taste but small quantity for the expensiv...,NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [9]:
rating_5 = groceries_df.loc[groceries_df['overall'] == 5]
rating_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
...,...,...,...,...,...,...,...
5074153,B01HJHSVG6,5,A1VL40O7D88AY0,"10 4, 2018",1538611200,Love this Vanilla. Blends and bakes beautifully.,NaN
5074154,B01HJHSVG6,5,A1LBAF0CNISB04,"10 4, 2018",1538611200,You'll never go back to plain ol' Vanilla Extr...,NaN
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [10]:
rating_1 = groceries_df.loc[groceries_df['overall'] == 1]
rating_1

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97.0
22,4639725183,1,AE937MVR5J8OL,"12 10, 2016",1481328000,Tasted old and not individually bag,104.0
27,4639725183,1,A27LF8222OIRRN,"09 10, 2016",1473465600,Not the great taste everyone was talking about :(,40.0
37,4639725183,1,A34RH95REHOQFO,"09 25, 2015",1443139200,Terrible. What a waste of money. This is not t...,1391.0
69,4639725043,1,A3ELDSSPP9RZPG,"03 31, 2017",1490918400,The tea is so fine it leaves tea particles flo...,194.0
...,...,...,...,...,...,...,...
5074110,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
5074111,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
5074112,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
5074120,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [11]:
rating_2 = groceries_df.loc[groceries_df['overall'] == 2]


In [12]:
rating_3 = groceries_df.loc[groceries_df['overall'] == 3]


In [13]:
rating_4 = groceries_df.loc[groceries_df['overall'] == 4]


In [14]:
groceries_df_12_45 = groceries_df[groceries_df.overall != 3]


In [15]:
#choosing every second row of the five star rating for a total of 600,000 to reduce its size from around 3.5 million ratings to a number closer to the 1 star ratings. 
rating_5_trimmed = rating_5[:1200000:2]
rating_5_trimmed

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
1659658,B005258A0K,5,AJQN9SH9FSPB1,"06 10, 2015",1433894400,Amazing breath freshener.\nDoes stick a little...,NaN
1659662,B005258A0K,5,AV37026DN7QFF,"06 6, 2015",1433548800,My daughter has started chewing gum to help ke...,NaN
1659667,B005258A0K,5,A2ID6N65Y5QYDT,"05 15, 2015",1431648000,This is the best xylitol gum on the market! L...,NaN
1659670,B005258A0K,5,A3V1S70XDFIPA,"05 7, 2015",1430956800,Nice to be able to get gum without aspartame i...,NaN


In [16]:
groceries_df_1_5 = [rating_5_trimmed, rating_1]
groceries_df_1_5
groc_1_5 = pd.concat(groceries_df_1_5)
groc_1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
5074110,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
5074111,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
5074112,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
5074120,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [17]:
g1_5 = pd.merge(rating_5_trimmed, rating_1, how='outer')
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
2,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
3,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
4,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
1005079,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
1005080,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
1005081,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
1005082,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [58]:
groceries_meta = pd.read_csv('Resources/fakemeat_meta.csv')
groceries_meta

,asin,brand,title,price
0,B000LKXRI6,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
1,B000LKZL4Y,Tofurky,"Turtle Island Tofurky Jurky, Ginger Teriyaki, ...",NaN
2,B000O6I9YY,Tofurky,"Tofurky, Deli Slices, Hickory Smoked, 5.5 oz",NaN
3,B000O6K870,Boca,Boca Original Chik'n Vegan Patties Made with N...,NaN
4,B000REIDYO,Quorn,"Quorn Chicken Style Tenders, 12 Oz (Pack of 12)",NaN
...,...,...,...,...
111,B01D9DHLNS,Simply Balanced,"Simply Balanced Fruit Strips Strawberry, Apric...",$25.99
112,B01DOAQ7KE,Tofurky,"Tofurky, Deli Slices, Bologna, 5.5 oz",NaN
113,B01DOB31XY,Tofurky,"Tofurky Slow Roasted Bbq Style Chick'n, 8 Ounc...",NaN
114,B01EVO3OAS,Tofurky,"Tofurky Veggie Burger Hearty Hemp, 10.5 Ounce ...",NaN


In [19]:
#groceries_meta = groceries_meta['price'].dropna()
#groceries_meta

In [20]:
#groceries_meta.count()

In [59]:
groceries15merged = pd.merge(g1_5, groceries_meta, on='asin', how='outer')
#groceries15merged = [g1_5, groceries_meta]
groceries15merged.head()


,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength,brand,title,price
0,1888861614,5.0,ALP49FBWT4I7V,"06 4, 2013",1.370304e+09,Very pleased with my purchase. Looks exactly l...,270.0,NaN,NaN,NaN
1,1888861614,5.0,A94ZG5CWN70E7,"12 1, 2013",1.385856e+09,Omg.. The S was inexpensive and EXACTLY what w...,22.0,NaN,NaN,NaN
2,1888861614,5.0,A1B65IWLUVOUQB,"09 6, 2013",1.378426e+09,This was exactly what we were looking for to m...,44.0,NaN,NaN,NaN
3,1888861614,1.0,ATQL0XOLZNHZ4,"02 21, 2014",1.392941e+09,"The S is beautiful, however upon checkout it i...",97.0,NaN,NaN,NaN
4,4639725183,5.0,A1QVBUH9E1V6I8,"11 19, 2014",1.416355e+09,No adverse comment.,257.0,NaN,NaN,NaN


In [67]:
groceries15merged_inner = pd.merge(g1_5, groceries_meta, on='asin', how='inner')
#groceries15merged_inner = [g1_5, groceries_meta]
groceries15merged_inner

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength,brand,title,price
0,B000LKXRI6,5,APYR8X25KSWMO,"07 2, 2007",1183334400,I love this stuff - you can't always find it i...,150.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
1,B000LKXRI6,5,A22V22KRTSNU19,"11 15, 2009",1258243200,This veggie jerky has great flavor. It is chew...,20.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
2,B000LKXRI6,5,A3LBG080QSSKQK,"04 2, 2009",1238630400,Super yummy! Best vegi jerky yet!!! I need to ...,114.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
3,B000LKXRI6,5,A35VJJW05NS0TH,"02 28, 2008",1204156800,This stuff is fantastic. I think it tastes pre...,37.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
4,B000LKXRI6,5,A34CSXOGVYF94S,"09 9, 2007",1189296000,"I'm addicted to this stuff, and love that Amaz...",581.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
5,B000REIDY4,5,AQKYOX1WA67V4,"03 14, 2017",1489449600,Quorn products are the best! I wish Amazon wou...,25.0,Quorn,"Quorn, Chicken Style Nuggets, 10.6 oz (Frozen)",NaN
6,B000REIDY4,5,APEA8F6H5UWM5,"08 18, 2016",1471478400,"I'm not the biggest fan of soy, which is the m...",443.0,Quorn,"Quorn, Chicken Style Nuggets, 10.6 oz (Frozen)",NaN
7,B000SKP4Y4,5,A15HB42N7AWHJI,"11 12, 2016",1478908800,Yummy! These have a very nice grilled/smoked h...,63.0,Boca,"Boca Original Vegan Veggie Burgers, 4 count Box",NaN
8,B000SKP4Y4,5,A1NUJZF8GWCJ14,"07 21, 2016",1469059200,This is sooo good!,265.0,Boca,"Boca Original Vegan Veggie Burgers, 4 count Box",NaN
9,B000VHRNY8,5,A26011Y16DUWV0,"06 20, 2016",1466380800,The pepper adds a nice flavor. I often bring ...,131.0,Tofurky,"Tofurky, Deli Slices, Peppered, 5.5 oz",NaN


In [69]:
groc15merged_inner = groceries15merged_inner[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength", 'brand', 'title']]
groc15merged_inner

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength,brand,title
0,B000LKXRI6,5,APYR8X25KSWMO,"07 2, 2007",1183334400,I love this stuff - you can't always find it i...,150.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce..."
1,B000LKXRI6,5,A22V22KRTSNU19,"11 15, 2009",1258243200,This veggie jerky has great flavor. It is chew...,20.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce..."
2,B000LKXRI6,5,A3LBG080QSSKQK,"04 2, 2009",1238630400,Super yummy! Best vegi jerky yet!!! I need to ...,114.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce..."
3,B000LKXRI6,5,A35VJJW05NS0TH,"02 28, 2008",1204156800,This stuff is fantastic. I think it tastes pre...,37.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce..."
4,B000LKXRI6,5,A34CSXOGVYF94S,"09 9, 2007",1189296000,"I'm addicted to this stuff, and love that Amaz...",581.0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce..."
5,B000REIDY4,5,AQKYOX1WA67V4,"03 14, 2017",1489449600,Quorn products are the best! I wish Amazon wou...,25.0,Quorn,"Quorn, Chicken Style Nuggets, 10.6 oz (Frozen)"
6,B000REIDY4,5,APEA8F6H5UWM5,"08 18, 2016",1471478400,"I'm not the biggest fan of soy, which is the m...",443.0,Quorn,"Quorn, Chicken Style Nuggets, 10.6 oz (Frozen)"
7,B000SKP4Y4,5,A15HB42N7AWHJI,"11 12, 2016",1478908800,Yummy! These have a very nice grilled/smoked h...,63.0,Boca,"Boca Original Vegan Veggie Burgers, 4 count Box"
8,B000SKP4Y4,5,A1NUJZF8GWCJ14,"07 21, 2016",1469059200,This is sooo good!,265.0,Boca,"Boca Original Vegan Veggie Burgers, 4 count Box"
9,B000VHRNY8,5,A26011Y16DUWV0,"06 20, 2016",1466380800,The pepper adds a nice flavor. I often bring ...,131.0,Tofurky,"Tofurky, Deli Slices, Peppered, 5.5 oz"


In [70]:
groceries_csv = groc15merged_inner.to_csv(path_or_buf='Resources/groceries_merged_trimmed_FM.csv', index=False)


In [51]:
unique_values = groc15merged_inner.brand.unique()
unique_values

array(['Tofurky', 'Quorn', 'Boca'], dtype=object)

In [82]:
# only fakemeat: ratings, words in review, date, price
fakemeat_df = groc15merged_inner[['brand','title','overall','reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
fakemeat_df

,brand,title,overall,reviewTime,unixReviewTime,reviewText,reviewlength
0,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",5,"07 2, 2007",1183334400,I love this stuff - you can't always find it i...,150.0
1,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",5,"11 15, 2009",1258243200,This veggie jerky has great flavor. It is chew...,20.0
2,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",5,"04 2, 2009",1238630400,Super yummy! Best vegi jerky yet!!! I need to ...,114.0
3,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",5,"02 28, 2008",1204156800,This stuff is fantastic. I think it tastes pre...,37.0
4,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",5,"09 9, 2007",1189296000,"I'm addicted to this stuff, and love that Amaz...",581.0
5,Quorn,"Quorn, Chicken Style Nuggets, 10.6 oz (Frozen)",5,"03 14, 2017",1489449600,Quorn products are the best! I wish Amazon wou...,25.0
6,Quorn,"Quorn, Chicken Style Nuggets, 10.6 oz (Frozen)",5,"08 18, 2016",1471478400,"I'm not the biggest fan of soy, which is the m...",443.0
7,Boca,"Boca Original Vegan Veggie Burgers, 4 count Box",5,"11 12, 2016",1478908800,Yummy! These have a very nice grilled/smoked h...,63.0
8,Boca,"Boca Original Vegan Veggie Burgers, 4 count Box",5,"07 21, 2016",1469059200,This is sooo good!,265.0
9,Tofurky,"Tofurky, Deli Slices, Peppered, 5.5 oz",5,"06 20, 2016",1466380800,The pepper adds a nice flavor. I often bring ...,131.0


In [83]:
fakemeat_df.count()

brand             12
title             12
overall           12
reviewTime        12
unixReviewTime    12
reviewText        12
reviewlength      12
dtype: int64

In [84]:
# sqlalchemy's create engine preparing the ability to connect to a sqlite database. then with .connect connecting to the database. 
engine = create_engine('sqlite:///groc15trimmed.sqlite', echo=True)
sqlite_connection = engine.connect()

In [85]:
#connecting to database and inputting info in from dataframe
sqlite_table = "groceriesALL"
ALLgroceries=g1_5.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-06-21 23:08:07,657 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-21 23:08:07,657 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:07,660 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-21 23:08:07,660 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:07,662 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-21 23:08:07,663 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:07,665 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesALL")
2021-06-21 23:08:07,666 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:07,669 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-21 23:08:07,670 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesALL',)
2021-06-21 23:08:07,672 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [86]:
MLgroceries_df = g1_5[['overall', 'reviewText']]
MLgroceries_df

,overall,reviewText
0,5,Very pleased with my purchase. Looks exactly l...
6,5,Omg.. The S was inexpensive and EXACTLY what w...
9,5,This was exactly what we were looking for to m...
11,5,No adverse comment.
13,5,Best black tea in US.\n\nHighly recommend.\nI ...
...,...,...
809605,1,not good.
809614,1,"When I received 2 bags of this icing, the bag ..."
809627,1,Ordered this by mistake thinking it was mering...
809636,1,I ordered 2 bags. One bag was completely slit ...


In [87]:
#connecting to database and inputting info in from dataframe
sqlite_table_2 = "groceriesMLData"
MLgroceries=MLgroceries_df.to_sql(sqlite_table_2, sqlite_connection, if_exists='replace')

2021-06-21 23:08:22,508 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesMLData")
2021-06-21 23:08:22,509 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:22,510 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesMLData")
2021-06-21 23:08:22,511 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:22,512 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-21 23:08:22,514 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:22,515 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesMLData")
2021-06-21 23:08:22,516 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:22,518 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-21 23:08:22,518 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesMLData',)
2021-06-21 23:08:22,520 INFO sqlalchemy.engine.Engine PRAGMA

In [ ]:
#FM_df = fakemeat_df[['brand','overall','reviewTime','unixReviewTime', 'reviewText', 'reviewlength']]
#FM_df

In [88]:
#connecting to database and inputting info in from dataframe
sqlite_table_3 = "fakemeatDF"
FakeMeat=fakemeat_df.to_sql(sqlite_table_3, sqlite_connection, if_exists='replace')

2021-06-21 23:08:32,603 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fakemeatDF")
2021-06-21 23:08:32,605 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:32,608 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fakemeatDF")
2021-06-21 23:08:32,609 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:32,612 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-21 23:08:32,613 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:32,616 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("fakemeatDF")
2021-06-21 23:08:32,617 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 23:08:32,622 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-21 23:08:32,624 INFO sqlalchemy.engine.Engine [raw sql] ('fakemeatDF',)
2021-06-21 23:08:32,627 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_li

In [81]:
#select and show data in groceries ALL table
#engine.execute('SELECT * FROM groceriesALL').fetchall()

In [80]:
#engine.execute('SELECT * FROM groceriesMLData').fetchall()

In [79]:
#engine.execute('SELECT * FROM fakemeatDF').fetchall()